In [ ]:
from crf.dataloader import MBStereo14, MBStereo14Unary
from torch.utils.data import DataLoader
from crf.mb_stereo_crf import logits2average_depth, CRFdepthRefiner
from crf.gaussian_matrix import FastGuidedFilter

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, torch
from oil.utils.utils import LoaderTo

In [ ]:
trainset=MBStereo14Unary(downsize=8)
device = torch.device('cuda')
dataloaders = {'train':DataLoader(trainset,batch_size=1,shuffle=True)}
dataloaders = {k:LoaderTo(v,device) for k,v in dataloaders.items()}

In [ ]:
for mb,i in zip(dataloaders['train'],range(3)):
    pass

In [ ]:
logits,imgrgb,features = mb[0]

In [ ]:
model = CRFdepthRefiner(r=20,niters=3,gamma=.1,eps=1e-3).cuda()

In [ ]:
plt.rcParams.update({'font.size': 18})
f, axarr = plt.subplots(1,3,figsize=(20,10))
L=60
a0 = axarr[0].imshow(logits2average_depth(mb[0][0])[0][0].cpu().data,cmap='bone',vmin=0,vmax=L)
a1 = axarr[1].imshow(model(mb[0])[0][0].cpu().data,cmap='bone',vmin=0,vmax=L)
a2 = axarr[2].imshow(imgrgb[0].permute(1,2,0).cpu().data.numpy())
# a3 = axarr[3].imshow(gt_depth/4,cmap='bone',vmin=0,vmax=L)
axarr[0].set_title("Unary")
axarr[1].set_title("DenseCRF")
#axarr[2].set_title("Tuned {} steps".format(num_steps))
#axarr[3].set_title("Ground Truth")

In [ ]:
plt.hist(logits2average_depth(mb[0][0])[0][0].cpu().data.numpy().reshape(-1),alpha=.5)
plt.hist(model(mb[0]).cpu().data.numpy().reshape(-1),alpha=.5)

In [ ]:
model(mb[0]).shape

In [ ]:
mb[0].shape

In [ ]:
%debug

In [ ]:
unaryds.show()

In [ ]:
def vis(img,unary_depth,target_depth,vmax=255):
    plt.rcParams.update({'font.size': 22})
    f, axarr = plt.subplots(1,3,figsize=(15,10))
    a0 = axarr[0].imshow(img)
    a1 = axarr[1].imshow(unary_depth,vmin=0,vmax=vmax)
    a2 = axarr[2].imshow(target_depth[...,0],vmin=0,vmax=vmax)
    axarr[0].set_title("Img")
    axarr[1].set_title("Unary")
    axarr[2].set_title("Ground Truth")
    plt.show()

for img, unary, target in zip(unaryds.left_imgs,unaryds.unary_logits,unaryds.left_target):
    #L = unary.shape[-1]
    print(img.shape,unary.shape,target.shape)
    #labels = np.arange(L)[None,None,:] #softmax(unary,axis=-1)*labels).sum(-1)
    unary_depth =np.argmax(unary,axis=-1)
    vis(img,unary_depth,target,512/unaryds.downsize)